<a href="https://colab.research.google.com/github/aadarshraghunathan/Neural-decision-making/blob/main/icnm_project_no_output.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%pylab inline 
import random 

Part 1: Setting up the neuron network 

In [ ]:
class MP_Neuron:
  def __init__(self,state,id):
    self.state = state
    self.id = id 

def update(neuron_id,theta): ### update for the ith neuron , theta is the global inhibition factor 
  global neuron_list,adj_mat,N
  update_val = 0
  weight_sum = 0 
  state_sum = 0 
  for j in range(N):
    update_val += (neuron_list[j].state)*adj_mat[neuron_id][j] 
    weight_sum += adj_mat[neuron_id][j]
    state_sum += neuron_list[j].state

  update_factor = update_val/weight_sum
  inhibit_factor = (state_sum/N)**2
  # print(inhibit_factor)
  inhibit_factor /= theta
  print(update_factor)
  print(inhibit_factor)
  if update_factor > inhibit_factor:
    neuron_list[neuron_id].state = 1
  else:
    neuron_list[neuron_id].state = 0 
  if(neuron_list_copy[neuron_id].state != neuron_list[neuron_id].state):
    print("Change")

    



In [ ]:
### creating all the neurons 
neuron_list = []
neuron_list_copy = []
N = 1000 ### number of neurons in the network
for i in range(N):
  ### randomise inital state, with 0.13(theta) probability of firing  
  rand_state =  random.uniform(0,1)
  if rand_state < 0.13 :
    state = 1 
  else :
    state = 0
  neuron = MP_Neuron(state,i)
  neuron_list.append(neuron)
  neuron_list_copy.append(neuron)
  


In [ ]:
### creating the graph between the neurons by making an adjacency matrix 
adj_mat = np.zeros((N,N), dtype = int )

### setting the connections with probability 0.36 (so far not considering attractor networks)
for i in range(N):
  for j in range(N):
    rand_connection = random.uniform(0,1)
    if rand_connection < 0.36:
      adj_mat[i][j] = 1 
    else :
      adj_mat[i][j] = 0 

print(adj_mat)

In [ ]:
### initialising the random time delays for updates 
### if neuron is active , use l1 = 0.07 
### if neuron is not active  , use l2 = 0.005

l1 = 0.07 
l2 = 0.005 

time_update = np.zeros((N))
for i in range(N):
  if(neuron_list[i].state == 1 ):
    t = np.random.exponential(scale = 1/l1)
    time_update[i]= int(t)
  else:
    t = np.random.exponential(scale = 1/l2)
    time_update[i] = int(t)
print(time_update)


In [ ]:
### updating all the neurons 
### after updating the value , update the values in the time_update array 
check_update = np.zeros(1000)
for t in range(100000):
  print(t)
  for i in range(N):
    if(time_update[i] == t):
      # print("yeet")
      update(i,1.3)
      check_update[i] += 1
      if(neuron_list[i].state == 1 ):
       x = np.random.exponential(scale = 1/l1)
       time_update[i] += int(x)
      else:
       x = np.random.exponential(scale = 1/l2)
       time_update[i] += int(x)


In [ ]:
print(check_update)

In [ ]:
for i in neuron_list:
  print(i.state, end= " ")
print("")
for j in neuron_list_copy:
  print(j.state, end = " ")

Part 2: Simulating the decision making process

In [ ]:
### functions to create the two attractor regions , and also update the graph weights when the attractor sets are created 
def create_attractor_sets():
  choice = [i for i in range(1000)]
  A = []
  B = []
  for i in range(100):
    picked = np.random.choice(choice) 
    A.append(picked)
    choice.remove(picked)

  for i in range(100):
    picked = np.random.choice(choice)
    B.append(picked)
    choice.remove(picked)
  
  return A,B

def update_weights(A,B):
  global adj_mat
  for i in range(100):
    for j in range(100):
      rand_connection = random.uniform(0,1)
      if rand_connection < 0.55:
        adj_mat[A[i]][A[j]] = 1 
        adj_mat[B[i]][B[j]] = 1 
      else :
        adj_mat[A[i]][A[j]] = 0 
        adj_mat[B[i]][B[j]] = 0 





In [ ]:
### create the attractor sets 
A,B = create_attractor_sets()
### update the graph weights 
update_weights(A,B)



In [ ]:
### function to create the two stimulus sets Sa and Sb 
### these sets are of random size in the range 1-100
### also function to update connections between the stimulus and the attractor sets 

def create_stimulus(A,B):
    choice = [i for i in range(1000)]
    for i in A:
      choice.remove(i)
    for j in B:
      choice.remove(j)
    size_SA = np.random.binomial(20,0.5)
    size_SB = np.random.binomial(20,0.5)
    SA = []
    SB = [] 
    for i in range(size_SA):
      picked = np.random.choice(choice)
      SA.append(picked)
      neuron_list[picked].state =1 
      choice.remove(picked)
    for i in range(size_SB):
      picked = np.random.choice(choice)
      SB.append(picked) 
      neuron_list[picked].state = 1 
      choice.remove(picked)

    return SA,SB

def update_weights_stimulus(A,B,SA,SB):
    global adj_mat
    for i in SA:
      for j in A:
        rand_connection = random.uniform(0,1)
        if rand_connection < 0.55:
          adj_mat[i][j] = 1 
        else :
          adj_mat[i][j] = 0 

    for i in SB:
      for j in B:
        rand_connection = random.uniform(0,1)
        if rand_connection < 0.55:
          adj_mat[i][j] = 1 
        else :
          adj_mat[i][j] = 0 



      


    

In [ ]:
### create the stimulus for the simulation 
SA,SB = create_stimulus(A,B)
### update all the connections between the stimulus set and the attractor set
update_weights_stimulus(A,B,SA,SB)
print(SA)
print(SB)

Now, we need to decide the winner. To do this , run 100000 updates and if one set A or B dominates the other for 500 consecutive steps, they are declared the winner .

In [ ]:
time_update = np.zeros((N))
for i in range(N):
  if(neuron_list[i].state == 1 ):
    t = np.random.exponential(scale = 1/l1)
    time_update[i]= int(t)
  else:
    t = np.random.exponential(scale = 1/l2)
    time_update[i] = int(t)

In [ ]:
### function to check which decision is winning 
### return 1 for A winning , 0 for no decision , -1 for B winning
def check_winner(A,B):
  sum_A = 0 
  sum_B = 0
  for i in A:
    sum_A += neuron_list[i].state
  for j in B:
    sum_B += neuron_list[j].state

  sum_A /= 100
  sum_B /= 100 
  print(sum_A)
  print(sum_B)
  if(sum_A - sum_B >= 0.75):
    return 1 
  elif(sum_B - sum_A >= 0.75):
    return -1
  else:
    return 0 



In [ ]:
### updating all the neurons 
### after updating the value , update the values in the time_update array

for t in range(100000):
  for i in range(N):
    if(time_update[i] == t):
      if(i in SA or i in SB):
        continue
      update(i,1)
      if(neuron_list[i].state == 1 ):
       t = np.random.exponential(scale = 1/l1)
       time_update[i] += int(t)
      else:
       t = np.random.exponential(scale = 1/l2)
       time_update[i] += int(t)
      
  if(check_winner(A,B) != 0 ):
    print("YAY")
  for i in A:
    print(neuron_list[i].state, end=" ")